In [5]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [6]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...


True

In [17]:
df = pd.read_csv('descriptions.csv')
df.head()

,Unnamed: 0,Synopsis,title
0,0,A cowboy doll is profoundly threatened and jea...,Toy Story
1,1,Four teenagers are sucked into a magical video...,Jumanji
2,2,John and Max resolve to save their beloved bai...,Grumpier Old Men
3,3,"Based on Terry McMillan's novel, this film fol...",Waiting to Exhale
4,4,George Banks must deal not only with his daugh...,Father of the Bride Part II


In [29]:
df.shape

(100, 2)

In [18]:
df = df.drop('Unnamed: 0', axis = 1)
df.head()

,Synopsis,title
0,A cowboy doll is profoundly threatened and jea...,Toy Story
1,Four teenagers are sucked into a magical video...,Jumanji
2,John and Max resolve to save their beloved bai...,Grumpier Old Men
3,"Based on Terry McMillan's novel, this film fol...",Waiting to Exhale
4,George Banks must deal not only with his daugh...,Father of the Bride Part II


In [19]:
def preprocess(text):
    text = re.sub(r'<.*?>', '', text)  # Supprimer les balises HTML
    text = re.sub(r'[^\w\s]', '', text)  # Supprimer la ponctuation
    tokens = word_tokenize(text)
    stop_words = set(stopwords.words('french'))  
    tokens = [word for word in tokens if word not in stop_words]

    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word.lower()) for word in tokens]

    return ' '.join(tokens)

In [20]:
df['Synopsis'] = df['Synopsis'].apply(preprocess)

In [21]:
df.head()

,Synopsis,title
0,a cowboy doll is profoundly threatened and jea...,Toy Story
1,four teenager are sucked into a magical video ...,Jumanji
2,john and max resolve to save their beloved bai...,Grumpier Old Men
3,based terry mcmillans novel this film follows ...,Waiting to Exhale
4,george bank must deal not only with his daught...,Father of the Bride Part II


In [22]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df['Synopsis'])

In [23]:
cosine = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [30]:
def get_recom(title, metrique=cosine):
    idx = df.index[df['title'] == title].tolist()[0]
    sim_scores = list(enumerate(metrique[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    film_indices = [i[0] for i in sim_scores]
    return df['title'].iloc[film_indices]


In [32]:
print(get_recom('Jumanji'))

65    Lawnmower Man 2: Beyond Cyberspace
36                Across the Sea of Time
22                             Assassins
8                           Sudden Death
21                               Copycat
89            The Journey of August King
76                             Nico Icon
26                          Now and Then
88                          Nick of Time
39              Cry, the Beloved Country
Name: title, dtype: object
